# init AD9174 for use with on board reference
This example uses the `HMC7044` and the 122.88 MHz reference oscillator on the `AD9174-FMC_EBZ` board to generate a DAC sampling clock. No external clock needed.

Requires litex_server running and connected to the VC707 USB-uart port:

```bash
litex_server --uart --uart-port /dev/ttyUSB0 --uart-baudrate 115200

```

In [1]:
import sys
from numpy import *
from time import sleep
from litex import RemoteClient
from ad9174 import Ad9174Settings, Ad9174Init

def getId(r):
    s = ""
    for i in range(64):
        temp = r.read(r.bases.identifier_mem + i * 4)
        if temp == 0:
            break
        s += chr(temp & 0xFF)
    return s

In [2]:
r = RemoteClient(csr_csv='../build/csr.csv', debug=False, port=1234)
r.open()
getId(r)

'AD9174 + VC707 test 2021-02-20 23:28:08'

In [3]:
settings = Ad9174Settings(json_file='../build/csr.json')
print(settings)

----------------
 JESD mode 20
----------------
INTERP_CH: 1  INTERP_MAIN: 1  DSP_CLK_DIV: 16
JESD204BSettings(): 5a 05 00 87 00 1f 00 0f 2f 23 80 00 00 e6 
         DID:  90        BID:   5     ADJCNT:   0        LID:   0 
       PHADJ:   0     ADJDIR:   0          L:   8        SCR:   1 
           F:   1          K:  32          M:   1          N:  16 
          CS:   0         NP:  16  SUBCLASSV:   1          S:   4 
       JESDV:   1         CF:   0         HD:   1       RES1:   0 
        RES2:   0       FCHK: 230 
   [ LINK_DW:  32     FR_CLK:   4 ]


In [4]:
ad = Ad9174Init(r, settings)
wr = ad.ad.wr
rr = ad.ad.rr

# 0. setup clocking
I was using the hmc7044 eval board windows software to generate the config file:
https://www.analog.com/en/products/hmc7044.html

In [5]:
# Adapter to run the config file ...
class Dut:
    def write(self, addr, val):
        ad.hmc.wr(addr, val)
dut = Dut()

# %run -i out12_122_8MHz.py
# %run -i 1228MHz_307MHz.py
# %run -i out12_368_64MHz.py
%run -i out12_245_76MHz.py

For clocking calculation see:

See https://docs.google.com/spreadsheets/d/1F6s6cVM1Lo6IOUgZoq9xm0ueGYkePZFeD96N0-kPR9o/edit#gid=2108895016

In [6]:
ad.fpga_print_clocks()

f_jesd = 245.757934 MHz  f_ref = 2.457579 MHz


# Everything below == untested

# 1. init AD9174

In [7]:
ad.init_ad9174()

AD917X_NVM_BLR_DONE: 1
PROD_ID: 0x9174
PROD_GRADE: 0  DEV_REVISION: 5
DAC PLL locked: 0


RuntimeError: DAC PLL not locked :(

# 3. Init the FPGA side

In [325]:
# r.regs.ctrl_reset.write(1)  # resets ALL clockdomains (HARSH!)
r.regs.control_control.write(0b01)  # resets PHYs and jesd core
print('status: {:03b}'.format(r.regs.control_status.read()))

# bit1: links_enable,  bit0: phys_reset
r.regs.control_control.write(0b10)
# bit2: /jsync,  bit1: links_ready,  bit0: phys_ready
print('status: {:03b}'.format(r.regs.control_status.read()))

ad.trigger_jref_sync()
ad.print_irq_flags(True, True)
if ad.print_irq_flags(True):
    print('😭')
else:
    print('😃')

status: 000
status: 000
SYNC_ROTATION_DONE 0


RuntimeError: Sync. of LMFC with JREF failed. JREF missing?

# 4. Test link

In [387]:
ad.print_irq_flags(True)
print()

ad.print_ilas()

ad.print_lane_status()
print()

ad.test_stpl()


JESD settings, received on lane 0 vs (programmed):
450: 5a (5a)
451: 05 (05)
452: 00 (00)
453: 82 (82)
454: 03 (03)
455: 1f (1f)
456: 05 (05)
457: 0f (0f)
458: 2f (2f)
459: 20 (20)
45a: 80 (80)
45b: 00 (00)
45c: 00 (00)
45d: e6 (e6)
CHK: e6 (e6) 

Lane status:
      LANE_DESKEW: 00000111
    BAD_DISPARITY: 00000000
     NOT_IN_TABLE: 00000000
 UNEXPECTED_KCHAR: 00000000
    CODE_GRP_SYNC: 00000111
       FRAME_SYNC: 00000111
    GOOD_CHECKSUM: 00000111
   INIT_LANE_SYNC: 00000111
FIFO_STATUS_REG_0: 00000000
FIFO_STATUS_REG_1: 00000000
fpga j_sync errs: 17456943

STPL test:
converter: 0, sample: 0, tp: 597a, fail: 0
converter: 1, sample: 0, tp: d27a, fail: 0
converter: 2, sample: 0, tp: 4b7a, fail: 0
converter: 3, sample: 0, tp: c47a, fail: 0
converter: 4, sample: 0, tp: 3d7a, fail: 0
converter: 5, sample: 0, tp: b67a, fail: 0


0

# Main DDS

In [35]:
# Setup DDSes
wr(0x1E6, (1 << 1))             # Enable DDSM_EN_CAL_DC_INPUT (see Fig. 80) (tone on / off)
wr(0x112, (1 << 3) | (1 << 2))  # Enable NCO + Modulus
wr(0x596, (1 << 3) | (1 << 2))  # Turn ON Transmit enable

# setup main DDS frequency and amplitude
ad.setTone(1, 1e9, 1, f_ref=5.12e9)

DC_CAL_TONE: ff 50 
DDSM_FTW: 00 00 00 00 00 32 
DDSM_FTW_LOAD_ACK: 1


In [36]:
# DC test-mode off, enable JESD input
ad.ad.wr(0x1e6, 0)

0

In [53]:
# Adjust DDS phase offset
ad.setTone(1, phase=0)

DDSM_NCO_PHASE_OFFSET: 00 00 


### Write IQ sample waveforms
kinda works. Output can be observed on IQ analyzer.

In [38]:
def write_samples(vals, m=0):
    ''' vals = [-1.0 ... 1.0], m = converter index '''
    bits = 16
    vals = (array(vals) * 0x7fff).astype(int)
    isNeg = vals < 0
    vals[isNeg] = (vals[isNeg] + (1 << bits)) & ((1 << bits) - 1)
    r.write(r.mems.m0_s0.base + 0x10000 * m, vals.tolist())
    r.regs.sample_gen_max_ind.write(len(vals) - 1)

In [43]:
write_samples(zeros(255), 0)
sleep(0.5)
write_samples(zeros(255), 1)

In [51]:
write_samples(linspace(-0.9, 0.9, 254), 0)

In [49]:
write_samples(sin(linspace(0, 3 * pi, 254)), 1)

In [50]:
write_samples(sin(linspace(0, 2.5 * pi, 254)), 0)

# app layer PRBS 😭
doesn't work 😠 this feature seems completely undocumented for AD9174 ☹️

In [60]:
r.regs.prbs_gen_sample_prbs_en.write(0)

In [12]:
r.regs.prbs_gen_sample_prbs_en.read()

0

In [79]:
ad.ad.help(0x14b)

reg 0x14b, PRBS:
    bit 7, PRBS_GOOD_Q, R, reset 0x00
    DAC1 good data indicator. 1: Correct PRBS sequence detected. 0: Incorrect sequence detected. Sticky; reset to 1 by PRBS_RESET.

    bit 6, PRBS_GOOD_I, R, reset 0x00
    DAC0 good data indicator. 0: Incorrect sequence detected. Sticky; reset to 1 by PRBS_RESET. 1: Correct PRBS sequence detected.

    bit 5, RESERVED, R, reset 0x00
    Reserved.

    bit 4, PRBS_INV_Q, R/W, reset 0x01
    DAC1 data inversion. 0: Expect normal data. 1: Expect inverted data.

    bit 3, PRBS_INV_I, R/W, reset 0x00
    DAC0 data inversion. 0: Expect normal data. 1: Expect inverted data.

    bit 2, PRBS_MODE, R/W, reset 0x00
    Select which PRBS polynomial is used for the datapath PRBS test. 0: 7-bit: x7 + x6 + 1. 1: 15-bit: x15 + x14 + 1.

    bit 1, PRBS_RESET, R/W, reset 0x00
    Reset error counters. 0: Normal operation. 1: Reset counters.

    bit 0, PRBS_EN, R/W, reset 0x00
    Enable PRBS checker. 0: Disable. 1: Enable.



In [10]:
ad.ad.wr(0x14b, 0b0111)
ad.ad.wr(0x14b, 0b0101)

for ch in range(7):
    ad.ad.wr(0x14e, ch)
    print('{:d}: {:08b}, {:3d}, {:3d}'.format(ch, ad.ad.rr(0x14b), ad.ad.rr(0x14c), ad.ad.rr(0x14d)))

0: 00000101, 255, 255
1: 00000101, 255, 255
2: 00000101, 255, 255
3: 11000101,   0,   0
4: 11000101,   0,   0
5: 11000101,   0,   0
6: 00000101, 255, 255
